In [1]:
from  __future__ import absolute_import
from __future__ import print_function
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
import tensorflow_hub as hub
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import logging
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_hub as hub
from PIL import Image
import pickle
import shutil
import matplotlib.pyplot as plt
import os
import random
import tensorflow as tf
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Model
from keras.applications import resnet
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import tensorflow_addons as tfa
from keras.regularizers import l2
from keras.activations import *
from DistanceLayer import DistanceLayer
from SiameseModel import SiameseModel
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print(tf.__version__)


batch_size = 19
batch_size_val = 89


2.8.0


In [2]:
features = {}
file = "resnet"
# load the features dictionary from the file
with open('features_' + file + '.pickle', 'rb') as handle:
    features = pickle.load(handle)

feature_dim = features["02461"].shape[0]

# Model

a lot of the code comes from https://keras.io/examples/vision/siamese_network/
and https://github.com/akarshzingade/image-similarity-deep-ranking/blob/master/deepRanking.py

In [3]:
anchor_input1 = Input(shape=(feature_dim,))
anchor_input2 = Input(shape=(feature_dim,))
left_input = Input(shape=(feature_dim,))
right_input = Input(shape=(feature_dim,))

comparator_left = Concatenate()([anchor_input1, left_input])
comparator_left = Dense(2048, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(comparator_left)
comparator_left = Dropout(0.3)(comparator_left)
comparator_left = Dense(2048, activation='linear', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(comparator_left)
comparator_left = Dropout(0.6)(comparator_left)

comparator_right = Concatenate()([anchor_input2, right_input])
comparator_right = Dense(2048, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(comparator_right)
comparator_right = Dropout(0.3)(comparator_right)
comparator_right = Dense(2048, activation='linear', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(comparator_right)
comparator_right = Dropout(0.3)(comparator_right)

c = Concatenate()([comparator_left, comparator_right])
c = Dense(2048, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(c)
c = Dropout(0.3)(c)
c = Dense(2048, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(c)
c = Dropout(0.3)(c)
c = Dense(1, activation='sigmoid')(c)

model = Model(inputs=[[anchor_input1,left_input],[anchor_input2,right_input]], outputs=c)

model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='binary_crossentropy',metrics=['accuracy'])

pred = model.predict(((np.random.rand(1,2048),np.random.rand(1,2048)),(np.random.rand(1,2048),np.random.rand(1,2048))))
print(pred)

model.summary()

[[0.41515496]]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 2048)]       0           []                               
                                                                               

In [4]:
# tf.keras.utils.plot_model(
#     model,
#     to_file="model.png",
#     show_shapes=True,
#     expand_nested=True,
    
# )

In [5]:
#convert the features dictionary to a tensorflow hash table
keys = tf.constant([key for key in features.keys()])
values = tf.constant([val for val in features.values()])

table = tf.lookup.experimental.DenseHashTable(
    value_dtype=tf.float32,
    key_dtype=tf.string,
    empty_key="empty_key",
    deleted_key="deleted_key",
    default_value=[-1]*feature_dim,
    )

table.insert(keys, values)

In [6]:
train_triplets = np.loadtxt('./train_triplets_split_shuffled.txt', dtype=str,delimiter = ' ')
val_triplets = np.loadtxt('./val_triplets_split_shuffled.txt', dtype=str,delimiter = ' ')
train_labels = np.loadtxt('./train_labels.txt', dtype=float,delimiter = ' ')
val_labels = np.loadtxt('./val_labels.txt', dtype=float,delimiter = ' ')
# train_dataset = tf.data.Dataset.from_tensor_slices(train_triplets)
train_dataset = tf.data.Dataset.from_tensor_slices((train_triplets,train_labels))
# val_dataset = tf.data.Dataset.from_tensor_slices(val_triplets)
val_dataset = tf.data.Dataset.from_tensor_slices((val_triplets,val_labels))
count = np.array(train_triplets).shape[0]
print(count)
print(np.array(val_triplets).shape[0])


@tf.function
def load_image(inputs,labels):
    filenames = inputs
    anchor = table[filenames[0]]
    positive = table[filenames[1]]
    negative = table[filenames[2]]
    output = (((anchor,positive),(anchor,negative)),labels)
    
    return output

train_dataset = train_dataset.map(load_image).cache().shuffle(buffer_size=1000).batch(batch_size).prefetch(20)
val_dataset = val_dataset.map(load_image).cache().batch(batch_size_val).prefetch(20)

7315
7476


In [7]:
# d = next(iter(train_dataset))
# d
# model.predict(d)



In [8]:
train_steps_per_epoch = int(count/batch_size)
# checkpoint_filepath = './checkpoints/checkpoint-{epoch}/'
#save checkpoint after every epoch
# callback = ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     # save_weights_only=True,
# )
callback = EarlyStopping(monitor='val_accuracy', patience=5)


model.fit(train_dataset, epochs=100,steps_per_epoch=train_steps_per_epoch,batch_size=batch_size,callbacks=[callback],validation_data=val_dataset)





Epoch 1/100
385/385 [==============================] - 9s 22ms/step - loss: 107.4120 - accuracy: 0.5105 - val_loss: 80.3748 - val_accuracy: 0.5730
Epoch 2/100
385/385 [==============================] - 8s 21ms/step - loss: 63.7795 - accuracy: 0.5491 - val_loss: 49.5506 - val_accuracy: 0.6247
Epoch 3/100
385/385 [==============================] - 8s 21ms/step - loss: 39.1705 - accuracy: 0.6477 - val_loss: 30.1593 - val_accuracy: 0.6937
Epoch 4/100
385/385 [==============================] - 8s 21ms/step - loss: 23.5508 - accuracy: 0.7085 - val_loss: 17.8814 - val_accuracy: 0.7092
Epoch 5/100
385/385 [==============================] - 8s 22ms/step - loss: 13.7879 - accuracy: 0.7385 - val_loss: 10.3729 - val_accuracy: 0.7072
Epoch 6/100
385/385 [==============================] - 8s 22ms/step - loss: 7.9267 - accuracy: 0.7416 - val_loss: 5.9729 - val_accuracy: 0.6989
Epoch 7/100
385/385 [==============================] - 8s 22ms/step - loss: 4.5348 - accuracy: 0.7541 - val_loss: 3.4959 - va

KeyboardInterrupt: 

# this code was used to initialize the distances file

In [ ]:
# #predict distances and save them to a file
# set_file = "val"
# dataset = np.loadtxt('./'+set_file+'_triplets_split_shuffled.txt', dtype=str,delimiter = ' ')
# print(dataset.shape)
# dataset = tf.data.Dataset.from_tensor_slices(dataset)
# size = batch_size_val if set_file == "val" else batch_size
# dataset = dataset.map(load_image).batch(size)
# data = []
# for triplet in iter(dataset):
#     anchor = triplet[0]
#     positive = triplet[1]
#     negative = triplet[2]
#     distances = model.predict([anchor,positive,negative])
#     for i in range(size):
#         data.append([distances[0][i],distances[1][i]])



# df = pd.DataFrame(data=data,columns=['anchor_left_distance_'+file,'anchor_right_distance_'+file])
# df.to_csv(set_file+'_distances.csv')




# append other distances

In [ ]:
#predict distances and save them to a file
set_file = "val"
dataset = np.loadtxt('./'+set_file+'_triplets_split_shuffled.txt', dtype=str,delimiter = ' ')
print(dataset.shape)
dataset = tf.data.Dataset.from_tensor_slices(dataset)
size = batch_size_val if set_file == "val" else batch_size
dataset = dataset.map(load_image).batch(size)
left = []
right = []
for triplet in iter(dataset):
    anchor = triplet[0]
    positive = triplet[1]
    negative = triplet[2]
    distances = model.predict([anchor,positive,negative])
    for i in range(size):
        left.append(distances[0][i])
        right.append(distances[1][i])



df = pd.read_csv(set_file+"_distances.csv",index_col=0)
df['anchor_left_distance_'+file] = left
df['anchor_right_distance_'+file] = right
df.to_csv(set_file+'_distances.csv')



(7476, 3)


TypeError: in user code:


    TypeError: tf__load_image() missing 1 required positional argument: 'labels'
